# Multiple Site Saturation Mutagenesis - User Notebook

In this notebook you can enter your own data to calculate primers.

## Setting up R

In [1]:
library("GoldenMutagenesis")

Loading required package: seqinr
Loading required package: stringr


## Setting up your data & parameters
Please modify this section according to your data.
### Input Sequence

In [2]:
input_sequence<-"ATGTCTCAGGTTCAGAGTGGCATTTTGCCAGAACATTGCCGCGCGGCGATTTGGATCGAAGCCAACGTGAAAGGGGAAGTTGACGCCCTGCGTGCGGCCAGTAAAACATTTGCCGACAAACTGGCAACTTTTGAAGCGAAATTCCCGGACGCGCATCTTGGTGCGGTGGTTGCCTTTGGTAACAACACCTGGCGCGCTCTGAGCGGCGGCGTTGGGGCAGAAGAGCTGAAAGATTTTCCGGGCTACGGTAAAGGCCTTGCGCCGACGACCCAGTTCGATGTGTTGATCCACATTCTTTCTCTGCGTCACGACGTAAACTTCTCTGTCGCCCAGGCGGCGATGGAAGCCTTTGGTGACTGCATTGAAGTGAAAGAAGAGATCCACGGCTTCCGTTGGGTTGAAGAGCGTGACCTGAGCGGCTTTGTTGACGGTACGGAAAACCCGGCGGGTGAAGAGACGCGTCGCGAAGTGGCGGTTATCAAAGACGGCGTGGATGCGGGCGGCAGCTATGTGTTTGTCCAGCGTTGGGAACACAACCTGAAGCAGCTCAACCGGATGAGCGTTCACGATCAGGAGATGGTGATCGGGCGCACCAAAGAGGCCAACGAAGAGATCGACGGCGACGAACGTCCGGAAACCTCTCACCTCACCCGCGTTGATCTGAAAGAAGATGGCAAAGGGCTGAAGATTGTTCGCCAGAGCCTGCCGTACGGCACTGCCAGTGGCACTCACGGTCTGTACTTCTGCGCCTACTGCGCGCGTCTGCATAACATTGAGCAGCAACTGCTGAGCATGTTTGGCGATACCGATGGTAAGCGTGATGCGATGTTGCGTTTCACCAAACCGGTAACCGGCGGCTATTATTTCGCACCGTCGCTGGACAAGTTGATGGCGCTGTAA"

### Envisioned Mutations

In [3]:
mutations<-c(137,143,147,232,234)
codon<-"NDT"
cuf<-"e_coli_316407.csv" #The Codon Usage Table which is used in the calculations

#### Available Codon Usage Tables

In [4]:
list_cu_table()

[1] "arabidopsis.csv"       "e_coli_316407.csv"     "s_cerevisiae_4932.csv"

### Further Parameters

In [5]:
calculate_level0<-T #Do you want to calculate primers in Level0?
#If you want to calcultate primers for Level2 only, enter calculate_level0<-F
#You can ignore the Level 0 parameters below in this case

############################
#####Level 2 parameters#####
###########################

#If you do not know how to set those parameters, just take the default value
recognition_site_lvl2<-"GGTCTC" #Recognition site sequence of the respective restriction enzyme
prefix="TT" #Spacer nucleotides matching the cleavage pattern of the enzyme
suffix="A" #Spacer nucleotides matching the cleavage pattern of the enzyme
vector=c("AATG", "AAGC") #Four basepair overhangs complementary to the created overhangs in the acceptor vector
replacement_range=5 #Maximum distance between two randomization sites to be incoporated into a single primer in amino acid residues
binding_min_length=4 #The minimal threshold value of the template binding sequence in amino acid residues
binding_max_length=9 #Maximal length of the binding sequence
target_temp=60 #Melting temperature of the binding sequence in °C
fragment_min_size=60 #Minimal size of a generated gene fragment in base pairs

############################
#####Level 0 parameters#####
###########################
recognition_site_lvl0<-"GAAGAC"
prefix_lvl0="TT"
suffix_lvl0="AA"
vector_lvl0=c("CTCA", "CTCG")


## Running the calculation
This chunk will create the output with the necessary primerset for you. 

### Domestication
This cell will calculate the primers for the domestication. Those are singe point mutations.

In [6]:
domest_mutations<-domesticate(input_sequence, recognition_site_lvl2 , cuf=cuf)
domest_mutations<-c(domest_mutations, domesticate(input_sequence, recognition_site_lvl0 , cuf=cuf))
if(length(domest_mutations) != 0) {
    domest_primerset<-mutate(input_sequence, replacements=domest_mutations, cuf=cuf)
    input_sequence<-domest_primerset@newsequence
    print_primer(primerset)
} 

[1] "No domestication needed."
[1] "No domestication needed."


### NDT Mutagenesis
This cell will calculate the primers for the Multiple Site Saturation Mutagenesis.

In [7]:
primers<-msd_mutate(input_sequence, prefix=prefix ,
                    restriction_enzyme=recognition_site_lvl2, suffix=suffix, 
                    vector=vector, replacements=mutations, 
                    replacement_range=replacement_range, 
                    binding_min_length=binding_min_length,
                    binding_max_length=binding_max_length, 
                    target_temp=target_temp, 
                    fragment_min_size=fragment_min_size )
if(calculate_level0==T) {
    primers<-primer_add_level(primers,  prefix=prefix_lvl0, 
                                   restriction_enzyme=recognition_site_lvl0,
                                   suffix=suffix_lvl0, vector=vector_lvl0)
}
print_primer(primers)


Fragment 1
Start 2, Stop 142, Length 141
Forward
TTGAAGACAACTCAAATGTCTCAGGTTCAGAGTGGCATTTTGCC
Temperature of binding site:  60.36446  °C 
Temperature difference:  0.3644563  K 
Reverse
TTGAAGACAACTCGACAAAGCCGCTCAGAHNACGCTCTTCAACCCAACGGAAGC
Temperature of binding site:  59.785  °C 
Temperature difference:  0.5794564  K 

Fragment 2
Start 143, Stop 231, Length 89
Forward
TTGAAGACAACTCATTGTTNDTGGTACGGAANDTCCGGCGGGTGAAGAGACGC
Temperature of binding site:  59.2169  °C 
Temperature difference:  0.7831005  K 
Reverse
TTGAAGACAACTCGAACAATCTTCAGCCCTTTGCCATCTTCTTT
Temperature of binding site:  59.04605  °C 
Temperature difference:  0.1708527  K 

Fragment 3
Start 232, Stop 300, Length 69
Forward
TTGAAGACAACTCATGTTNDTCAGNDTCTGCCGTACGGCACTGCCAG
Temperature of binding site:  60.52991  °C 
Temperature difference:  0.5299114  K 
Reverse
TTGAAGACAACTCGAAGCTTACAGCGCCATCAACTTGTCCAGC
Temperature of binding site:  60.90327  °C 
Temperature difference:  0.3733614  K 

Input Sequence:
 ATGTCTCAGGTTCAGAGTGGC